In [1]:
import vapor
import anndata as ad
from vapor.config import VAPORConfig

/home/noahcohenkalafut/miniconda3/envs/vapor-env/lib/python3.10/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)
/home/noahcohenkalafut/miniconda3/envs/vapor-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load data
adata = ad.read_h5ad('./data/drugSeries.h5ad')

# Filter data
valid_treatments = ['DMSO_48hr']
valid_treatments += [f'Tram_{i}hr' for i in (3, 6, 12, 24, 48)]  # Maybe hold out 24?
mask = adata.obs['treatment'].isin(valid_treatments)
adata = adata[mask]

# Annotate timepoints
timepoint_dict = {k: v for k, v in zip(valid_treatments, (0, 3, 6, 12, 24, 48))}  # Should this be scaled?
adata.obs['Timepoint'] = adata.obs['treatment'].apply(lambda x: timepoint_dict[x])

In [32]:
# Create supervised dataset
dataset = vapor.dataset_from_adata(
    adata,
    root_where=['treatment=DMSO_48hr'],
    terminal_where=['treatment=Tram_48hr'],
    time_label='Timepoint',
    root_n=200, terminal_n=200, seed=42,
    scale=True)

Time Range: (0.000, 1.000)
Data scaled per row.
Root selection: matched=2256, sampled=200, where={'treatment': 'DMSO_48hr'}
Terminal selection: matched=1439, sampled=200, where={'treatment': 'Tram_48hr'}


In [ ]:
# Config
config = VAPORConfig(
    latent_dim=64, n_dynamics=10, lr=5e-4,
    beta=0.01, eta=1.0, t_max=4, epochs=500,
    batch_size=512)

# Initialize model
model = vapor.initialize_model(adata.n_vars, config=config)

Initializing model:
  Input dim: 32738
  Latent dim: 64
  Encoder dims: [2048, 512, 128]
  Decoder dims: [128, 512, 2048]
  N dynamics: 10
0.75


In [ ]:
# Train
trained_model = vapor.train_model(model, dataset, config=config)

Training on device: None
Training for 500 epochs with batch size 512
Data split: train=6598, test=1650 (test_size=0.2)

Starting training...
--------------------------------------------------------------------------------
Epoch   1/500 | Time: 50.37s | Recon: 2.0059 | KL: 0.1201 | Traj: 1.8653 | Prior: 0.0609 | Psi: 0.9986
Epoch   2/500 | Time: 47.80s | Recon: 1.9953 | KL: 0.1199 | Traj: 1.8351 | Prior: 0.0496 | Psi: 0.9931
Epoch   3/500 | Time: 52.86s | Recon: 1.9850 | KL: 0.1197 | Traj: 1.8879 | Prior: 0.0512 | Psi: 0.9864
Epoch   4/500 | Time: 56.50s | Recon: 1.9751 | KL: 0.1195 | Traj: 1.8431 | Prior: 0.0382 | Psi: 0.9775
Epoch   5/500 | Time: 49.41s | Recon: 1.9653 | KL: 0.1194 | Traj: 1.8493 | Prior: 0.0387 | Psi: 0.9673
Epoch   6/500 | Time: 50.90s | Recon: 1.9556 | KL: 0.1193 | Traj: 1.8483 | Prior: 0.0328 | Psi: 0.9582
Epoch   7/500 | Time: 49.55s | Recon: 1.9464 | KL: 0.1191 | Traj: 1.8725 | Prior: 0.0268 | Psi: 0.9481
Epoch   8/500 | Time: 48.75s | Recon: 1.9374 | KL: 0.1191

In [ ]:
trained_model.save

In [ ]:
trained_model.load

In [ ]:
# USE